In [1]:
# code and data: https://github.com/rpeden/cat-or-not/releases
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from PIL import Image
from random import shuffle, choice
import numpy as np
import os

In [2]:
IMAGE_SIZE = 256
IMAGE_DIRECTORY = './data/training_set'

In [3]:
def label_img(name):
    if name == 'cats': return np.array([1, 0])
    elif name == 'notcats' : return np.array([0, 1])


In [4]:
def load_data():
    print("Loading images...")
    train_data = []
    directories = next(os.walk(IMAGE_DIRECTORY))[1]

    for dirname in directories:
        print("Loading {0}".format(dirname))
        file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, dirname)))[2]
        for i in range(len(file_names)): # len(file_names)
            image_name = choice(file_names)
            image_path = os.path.join(IMAGE_DIRECTORY, dirname, image_name)
            label = label_img(dirname)
            if "DS_Store" not in image_path:
                img = Image.open(image_path)
                img = img.convert('L')
                img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
                train_data.append([np.array(img)/255, label])
    return train_data

In [5]:
def create_model():
    model = Sequential()
    model.add(Input(IMAGE_SIZE, IMAGE_SIZE, 1))
    model.add(Conv2D(32, kernel_size = (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation = 'softmax'))

    return model

In [6]:
training_data = load_data()
training_images = np.array([i[0] for i in training_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
training_labels = np.array([i[1] for i in training_data])

Loading images...
Loading cats
Loading notcats


In [7]:
print('creating model')
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('training model')
model.fit(training_images, training_labels, batch_size=50, epochs=10, verbose=1)
model.save("model.h5")

creating model


C:\Users\mikec\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


training model
Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 263s 2s/step - accuracy: 0.7685 - loss: 0.5285
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.8784 - loss: 0.3143
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9221 - loss: 0.2072
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9513 - loss: 0.1403
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.9630 - loss: 0.1003
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.9725 - loss: 0.0817
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.9793 - loss: 0.0575
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.9836 - loss: 0.0474
Epoch 9/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.9920 - loss: 0.0261
Epoch 10/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.9880 - loss: 0.0358


In [8]:
IMAGE_DIRECTORY = './data/test_set'
test_data = load_data()
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
test_labels = np.array([i[1] for i in test_data])

Loading images...
Loading cats
Loading notcats


In [9]:
test_images.shape

(2666, 256, 256, 1)

In [10]:
test_labels.shape, training_labels.shape

((2666, 2), (5669, 2))

In [11]:
loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print("accuracy: {0}".format(acc * 100))

84/84 ━━━━━━━━━━━━━━━━━━━━ 28s 329ms/step - accuracy: 0.8355 - loss: 0.8068
accuracy: 79.29482460021973
